<a href="https://colab.research.google.com/github/git-zeno/LearnMate-PDF-QA/blob/main/genAI_ibm_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers
!pip install -U pdfplumber
!pip install -U ibm-watsonx-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import os

os.makedirs("utils", exist_ok=True)

with open("utils/pdf_utils.py", "w") as f:
    f.write('''
import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            content = page.extract_text()
            if content:
                text += content + "\\n"
    return text
''')

with open("utils/text_splitter.py", "w") as f:
    f.write('''
def split_text(text, max_chunk_size=300):
    paragraphs = text.split("\\n")
    chunks = []
    current_chunk = ""

    for para in paragraphs:
        if len(current_chunk) + len(para) <= max_chunk_size:
            current_chunk += para + "\\n"
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para + "\\n"

    if current_chunk.strip():
        chunks.append(current_chunk.strip())

    return chunks
''')

In [ ]:
from utils.pdf_utils import extract_text_from_pdf
from utils.text_splitter import split_text
from sentence_transformers import SentenceTransformer, util
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
import torch

# Load models
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# IBM watsonx credentials (default, feel free to change)
api_key = "vBVJUXhWG38XeIyiQs6OUnWY4_R8KIjdHvB4tsUg6rIC"
project_id = "cf0c115c-5e77-471c-aede-f35fc523835c"

creds = Credentials(api_key=api_key, url="https://us-south.ml.cloud.ibm.com")

qa_model = ModelInference(
    model_id="ibm/granite-13b-instruct-v2",
    credentials=creds,
    project_id=project_id,
    params={
        "temperature": 0.7,
        "max_new_tokens": 600,
        "decoding_method": "sample",
        "top_k": 100,
        "top_p": 0.95,
        "repetition_penalty": 1.2
    }
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


In [ ]:
from google.colab import files

print("Upload a PDF file")
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
raw_text = extract_text_from_pdf(pdf_path)

chunks = split_text(raw_text, max_chunk_size=300)
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

print(f"Loaded! Total chunks: {len(chunks)}")

Upload a PDF file


Saving note0.pdf to note0.pdf
Loaded! Total chunks: 18


In [ ]:
def ask_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)
    similarities = util.cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]
    score = similarities[best_idx].item()

    print(f"\nMatch confidence: {score * 100:.2f}%")

    if score < 0.45:
        print("Answer: Sorry, I couldn't find that information in the document.")
        return

    prompt = f"""
You are an expert teaching assistant. Your job is to help students deeply understand topics using the provided study material.
Answer the question in a clear, detailed, and informative manner, covering all relevant concepts from the context.
Include examples or elaborations if possible. If the answer is not found in the context, say: "Sorry, I couldn't find that information in the document."
Only answer based on the given context. Do not make up facts.

Context:
{best_chunk}

Question: {question}
Answer:
"""

    response = qa_model.generate(prompt=prompt)
    answer = response['results'][0]['generated_text'].strip()
    print(f"Answer:\n ->{answer}")


In [ ]:
while True:
    q = input("\nQ: Ask a question (or type 'exit'): ").strip()
    if q.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    ask_question(q)


Q: Ask a question (or type 'exit'): what is ai

Match confidence: 81.28%
Answer:
 ->field of computer science

Q: Ask a question (or type 'exit'): what is tree

Match confidence: 13.83%
Answer: Sorry, I couldn't find that information in the document.

Q: Ask a question (or type 'exit'): exit
Goodbye!
